In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

2024-03-15 15:23:58.300620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 15:23:58.300816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 15:23:58.466394: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
display(df.head())
print(df.shape)

/tmp/ipykernel_1062/3152433989.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])


,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


(54214, 4)


import re
def clean_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [5]:
df[0:1]

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...


In [6]:
df.drop(columns=['ID','TITLE'],inplace=True)

In [7]:
df.head()

,GENRE,DESCRIPTION
0,drama,Listening in to a conversation between his do...
1,thriller,A brother and sister with a past incestuous r...
2,adult,As the bus empties the students for their fie...
3,drama,To help their unemployed father make ends mee...
4,drama,The film's title refers not only to the un-re...


In [8]:
average_length = df['DESCRIPTION'].str.len().mean()

print(f"The average length of movie descriptions is: {average_length:.2f}")


The average length of movie descriptions is: 600.45


In [9]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
df['DESCRIPTION']=df['DESCRIPTION'].str.lower()

In [11]:
df.head()

,GENRE,DESCRIPTION
0,drama,listening in to a conversation between his do...
1,thriller,a brother and sister with a past incestuous r...
2,adult,as the bus empties the students for their fie...
3,drama,to help their unemployed father make ends mee...
4,drama,the film's title refers not only to the un-re...


In [12]:
import re
def clean_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)
    return text
df['DESCRIPTION']=df['DESCRIPTION'].apply(clean_text)
df['DESCRIPTION']

0         listening in to a conversation between his do...
1         a brother and sister with a past incestuous r...
2         as the bus empties the students for their fie...
3         to help their unemployed father make ends mee...
4         the films title refers not only to the unreco...
                               ...                        
54209     this shortlived nbc live sitcom centered on b...
54210     the next generation of exploitation the siste...
54211     ze bestaan echt is a standup comedy about gro...
54212     walter and vivian live in the country and hav...
54213     on labor day weekend 1935 the most intense hu...
Name: DESCRIPTION, Length: 54214, dtype: object

In [13]:
# from nltk.corpus import stopwords

# def remove_stopwords(text):
#     stop_words = set(stopwords.words('english'))
#     new_text = [word for word in text.split() if word not in stop_words]
#     return ' '.join(new_text)
# df['DESCRIPTION']=df['DESCRIPTION'].apply(remove_stopwords)

In [14]:
# import spacy 
# nlp=spacy.load('en_core_web_sm')
# import re 
# def tokenize(text):
#     tokens=re.split('\W+',text)
#     return tokens
# df['DESCRIPTION']=df['DESCRIPTION'].apply(tokenize)
# df['DESCRIPTION'].head()

In [15]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def steam_words(text):
    return " ".join([ps.stem(word) for word in text])
df['DESCRIPTION'].apply(steam_words)

0          l i s t e n i n g   i n   t o   a   c o n v ...
1          a   b r o t h e r   a n d   s i s t e r   w ...
2          a s   t h e   b u s   e m p t i e s   t h e ...
3          t o   h e l p   t h e i r   u n e m p l o y ...
4          t h e   f i l m s   t i t l e   r e f e r s ...
                               ...                        
54209      t h i s   s h o r t l i v e d   n b c   l i ...
54210      t h e   n e x t   g e n e r a t i o n   o f ...
54211      z e   b e s t a a n   e c h t   i s   a   s ...
54212      w a l t e r   a n d   v i v i a n   l i v e ...
54213      o n   l a b o r   d a y   w e e k e n d   1 ...
Name: DESCRIPTION, Length: 54214, dtype: object

In [16]:
df['DESCRIPTION']

0         listening in to a conversation between his do...
1         a brother and sister with a past incestuous r...
2         as the bus empties the students for their fie...
3         to help their unemployed father make ends mee...
4         the films title refers not only to the unreco...
                               ...                        
54209     this shortlived nbc live sitcom centered on b...
54210     the next generation of exploitation the siste...
54211     ze bestaan echt is a standup comedy about gro...
54212     walter and vivian live in the country and hav...
54213     on labor day weekend 1935 the most intense hu...
Name: DESCRIPTION, Length: 54214, dtype: object

In [17]:
df.head()

,GENRE,DESCRIPTION
0,drama,listening in to a conversation between his do...
1,thriller,a brother and sister with a past incestuous r...
2,adult,as the bus empties the students for their fie...
3,drama,to help their unemployed father make ends mee...
4,drama,the films title refers not only to the unreco...


In [18]:
X=df['DESCRIPTION']
y=df['GENRE']

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
count= CountVectorizer(stop_words='english')
X_vec=count.fit_transform(X)
tfidf = TfidfTransformer()
X_tfidf=tfidf.fit_transform(X_vec)
#X_vectorized=vectorizer.todense()
#_train_vectorized = vectorizer.fit_transform(X_train)
#X_train_vectorized=X_train_vectorized.todense()
#rint(X_train_vectorized)
#_test_vectorized = vectorizer.transform(X_test)
#X_test=X_test_vectorized.todense()

In [20]:
X_train,X_test,y_train,y_test=train_test_split(X_tfidf,y,test_size=0.2,random_state=42)

In [21]:
X_train

<43371x150608 sparse matrix of type '<class 'numpy.float64'>'
	with 1971800 stored elements in Compressed Sparse Row format>

In [22]:
X_test

<10843x150608 sparse matrix of type '<class 'numpy.float64'>'
	with 493949 stored elements in Compressed Sparse Row format>

In [ ]:
X_train_dense = X_train.toarray()

# Now convert the dense array to a SparseTensor
X_train_sparse = tf.sparse.from_dense(X_train_dense)

# Now reorder the SparseTensor
X_train_vectorized_ordered = tf.sparse.reorder(X_train_sparse)

In [ ]:
X_test_dense = X_test_vectorized.toarray()

# Now convert the dense array to a SparseTensor
X_test_sparse = tf.sparse.from_dense(X_test_dense)

# Now reorder the SparseTensor
X_test_vectorized_ordered = tf.sparse.reorder(X_test_sparse)

In [ ]:
from tensorflow.python.framework.sparse_tensor import SparseTensor
print(isinstance(X_train, SparseTensor))

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y_train_label=label_encoder.fit_transform(y_train)
y_test_label=label_encoder.transform(y_test)

In [ ]:
model = Sequential([
    Dense(512, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.8),
    Dense(256, activation='relu'),
    # Dropout(0.8),
    Dense(128, activation='relu'),
    # Dropout(0.8),

    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(27, activation='softmax')
])



optimizer = Adam(learning_rate=0.003)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_label, epochs=15, batch_size=64, validation_data=(X_test, y_test_label))
# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(X_test, y_test_label)

print(f'Test Accuracy: {test_accuracy}')
print(f'Test Loss: {test_loss}')